In [10]:
import json
import requests
import threading
import time
import os
from seleniumwire import webdriver
from selenium.webdriver.common.proxy import Proxy, ProxyType

In [12]:
def interceptor(request):
    if request.url.startswith('https://us-central1-popxi-f3a4d.cloudfunctions.net/stats?count='):
        params = request.params
        params['count'] = '5000'
        request.params = params
        print('Popping...')

In [13]:
def initBrowser(proxy = None):
    options = webdriver.ChromeOptions()
    options.add_argument('ignore-certificate-errors')
    #options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    #options.add_argument("disable-gpu")
    options.add_argument("--mute-audio")
    #options.add_argument("--disable-gpu")
    #seleniumwire_options = {
    #    'enable_har': True  # Capture HAR data, retrieve with driver.har
    #}
    #driver = webdriver.Chrome('chromedriver', options=options, seleniumwire_options=seleniumwire_options)
    if proxy is not None:
        chrome_options.add_argument('--proxy-server=%s' % PROXY)
    
    driver = webdriver.Chrome('chromedriver', options=options)
    driver.request_interceptor = interceptor
    #driver.scopes = [
    #    '.*www.google.com/*',
    #    '.*us-central1-popxi-f3a4d.cloudfunctions.net/stats*.*'
    #]
    driver.get('https://popxi.click/')
    driver.execute_script('var event=new KeyboardEvent("keydown",{key:"g",ctrlKey:!0});setInterval(function(){for(i=0;i<1;i++)document.dispatchEvent(event)},200);')
    return driver

In [15]:
def getRequests(driver):
    get = False
    while not get:
        for req in driver.requests:
            if req.url.startswith('https://us-central1-popxi-f3a4d.cloudfunctions.net/stats?count='):
                try:
                    print('Response: ' + str(req.response.status_code))
                except:
                    print('Response: None')
                print('Deleting cookies...')
                driver.delete_all_cookies()
                del driver.requests
                driver.execute_script("window.open('https://popxi.click/');")
                driver.switch_to.window(driver.window_handles[0])
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
                print('Deleted.')
                time.sleep(2)
                driver.execute_script('var event=new KeyboardEvent("keydown",{key:"g",ctrlKey:!0});setInterval(function(){for(i=0;i<1;i++)document.dispatchEvent(event)},200);')
        time.sleep(1)

In [16]:
def Run(proxy = None):
    print("Starting browser...")
    driver = initBrowser(proxy)
    print("Browser Started.")
    print("Fetching requests...")
    getRequests(driver)

In [15]:
Run()